In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2 as cv
import os
import numpy as np
import glob
import imutils
import matplotlib.pyplot as plt

In [ ]:
path = '/content/drive/MyDrive/Matched1/'

In [ ]:
data_path = os.path.join(path, '*jpeg')
files = glob.glob(data_path)
files = sorted(files)
data = []
for f1 in files:
    #print(f1)
    img = cv.imread(f1)
    #img = cv.rotate(img, cv.cv.ROTATE_90_CLOCKWISE)
    data.append(img)

In [ ]:
gray = []
for f2 in data:
    img_gry = cv.cvtColor(f2,cv.COLOR_BGR2GRAY)
    gray.append(img_gry)

In [ ]:
print(cv.__version__)

4.4.0


In [ ]:
!pip install opencv-contrib-python==4.4.0.44

     |████████████████████████████████| 55.7MB 66kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [ ]:
# create SIFT feature extractor
sift = cv.xfeatures2d.SIFT_create()

In [ ]:
keyp = list()
descr = list()
print(len(keyp))
print(len(descr))

0
0


In [ ]:
for f3 in gray:
    a, b = sift.detectAndCompute(f3, None)
    keyp.append(a)
    descr.append(b)

In [ ]:
bf = cv.BFMatcher(cv.NORM_L2, crossCheck=False)

In [ ]:
mtch = list()
print(len(mtch))
tmp = list()

0


In [ ]:
mtch = []
j1 = 0
while j1 < (len(descr) - 1):
    a = descr[j1]
    b = descr[j1 + 1]
    c = bf.knnMatch(a,b,2)
    matches = []

    # loop over the raw matches
    for m,n in c:
        # ensure the distance is within a certain ratio of each
        # other (i.e. Lowe's ratio test)
        if m.distance < n.distance * 0.75:
            matches.append(m)
    mtch.append(matches)
    j1 = j1 + 2

In [ ]:
#List of matched images
mtch_img = list()
print(len(mtch_img))
print(len(gray))

0
5


In [ ]:
k = 0
j = 0
while j < (len(data) - 1):
    m = cv.drawMatches(data[j+1], keyp[j], data[j], keyp[j+1], mtch[k][:100], None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    mtch_img.append(m)
    j = j + 2
    k = k + 1

In [ ]:
#Apply Homography
i = 0
j = 0
k = 0
M = []
homo = []
while i < (len(keyp) - 1):
    j = i + 1
    a = np.float32([kp.pt for kp in keyp[i]])
    b = np.float32([kp.pt for kp in keyp[j]])
    #if len(mtch[k]) > 4:
    pts_a = np.float32([a[m.queryIdx] for m in mtch[k]])
    pts_b = np.float32([b[m.trainIdx] for m in mtch[k]])
    H, status = cv.findHomography(pts_a, pts_b, cv.RANSAC, 4)
    M.append((mtch[k], H, status))
    homo.append(H)
    #else:
        #homo.append(None)
    i = i + 2
    k = k + 1

In [ ]:
for H in homo:
    print(H)

[[ 1.00007760e+00  9.26428022e-06 -8.88072053e+02]
 [-1.62972862e-07  1.00008303e+00  3.99003142e+00]
 [ 4.76383600e-08  3.07219331e-08  1.00000000e+00]]
[[ 9.50495563e-01 -4.64791486e-02 -8.19480342e+02]
 [ 6.00765335e-02  7.67688585e-01  8.05279762e+01]
 [ 5.57123163e-05 -1.60392660e-04  1.00000000e+00]]


In [ ]:
# Apply panorama correction
res = []
i = 0
k = 0
while i < (len(data) - 1):
    w = data[i+1].shape[1] + data[i].shape[1]
    h = data[i+1].shape[0] + data[i].shape[0]
    r = cv.warpPerspective(data[i+1], homo[k], (w,h))
    r[0:data[i].shape[0], 0:data[i].shape[1]] = data[i]
    res.append(r)
    i = i + 2
    k = k + 1

In [ ]:
res_final = []
for r in res:
    # transform the panorama image to grayscale and threshold it 
    g = cv.cvtColor(r, cv.COLOR_BGR2GRAY)
    thresh = cv.threshold(g, 0, 255, cv.THRESH_BINARY)[1]

    # Finds contours from the binary image
    cnts = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # get the maximum contour area
    c = max(cnts, key=cv.contourArea)

    # get a bbox from the contour area
    (x, y, w, h) = cv.boundingRect(c)

    # crop the image to the bbox coordinates
    r1 = r[y:y + h, x:x + w]
    res_final.append(r1)

In [ ]:
p1 = "/content/drive/MyDrive/Matched1/"
#os.mkdir(p1)

In [ ]:
res_final1 = []
for r in res_final:
    result = cv.rotate(r, cv2.ROTATE_90_COUNTERCLOCKWISE)
    plt.imshow(result)
    res_final1.append(result)

In [ ]:
d = int(0)
for f4 in res_final:
    cv.imwrite(p1+"matched_"+str(d)+".jpeg", f4)
    d = d + 1